In [27]:
import skimage.io as io
import time
import numpy as np
import cv2
import os

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from preprocessing import *
from LocalBinaryPattern import *
from ChainCodesFeatures import *
from PolygonFeatures import * 
from WindowFeatures import *
from windows import *

In [28]:
def Feature_Extractor(img):
    
    
    
    img = cutHandWriting(img)
    
    img = cv2.resize(img,(img.shape[0]//4,img.shape[1]//4))
    
    lbp = LocalBinaryPatterns(255,16)
    lbp_features = lbp.describe(img)
    #st = time.time()
    edged = cv2.Canny(img, 30, 200)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #en = time.time()
#     print("Contours : ",(en-st)*1000,"ms")
    
    #st = time.time()
    windows, clusters, clusteredWindows = getClusteredWindows(img,13,10)
    #en = time.time()
#     print("Kesho : ",(en-st)*1000,"ms")
    
    #st = time.time()
    F1,F2,F3,F4,F5,F7,F8,F9 = Chain_Codes_Feature_Extractor(img,contours,clusteredWindows)
    #en = time.time()
#     print("Ramzy : ",(en-st)*1000,"ms")
    
    #st = time.time()
    f10, f11, f12, f13, f14 = PolygonFeatures(contours)
    #en = time.time()
#     print("Ibrahim Lefta : ",(en-st)*1000,"ms")
    
    #st = time.time()
    Windows = list()
    
    #for i in range(clusteredWindows.shape[0]):
        #print(clusteredWindows[i])
        #Windows  += list(WindowFeatures(clusteredWindows[i].astype(np.uint8)))
    #en = time.time()
#     print("October : ",(en-st)*1000,"ms")
    
    Features = list(F1)+list(F2)+list(F3)+list(F4)+list(F5)+list(F7)+list(F8)+list(F9)+list(f10)+list(f11)+list(f12)+list(f13)+list(f14)+list(lbp_features)+Windows
    return Features
    

In [29]:
correct = []
f = open("correct.txt", "r")
for x in f:
    correct.append(int(x)) 

In [30]:
test_cases = os.listdir("TestData")
test_cases.sort()

In [ ]:
i = 0 
acc = 0
for case in test_cases:
    writer1_1  = cv2.imread('TestData/'+case+"/1/1.png", cv2.IMREAD_GRAYSCALE)
    writer1_2  = cv2.imread('TestData/'+case+"/1/2.png", cv2.IMREAD_GRAYSCALE)
    writer2_1  = cv2.imread('TestData/'+case+"/2/1.png", cv2.IMREAD_GRAYSCALE)
    writer2_2  = cv2.imread('TestData/'+case+"/2/2.png", cv2.IMREAD_GRAYSCALE)
    writer3_1  = cv2.imread('TestData/'+case+"/3/1.png", cv2.IMREAD_GRAYSCALE)
    writer3_2  = cv2.imread('TestData/'+case+"/3/2.png", cv2.IMREAD_GRAYSCALE)
    test_img =  cv2.imread('TestData/'+case+"/test.png", cv2.IMREAD_GRAYSCALE)

    st = time.time()
    
    f1_1  = Feature_Extractor(writer1_1)
    f1_2  = Feature_Extractor(writer1_2)
    f2_1  = Feature_Extractor(writer2_1)
    f2_2  = Feature_Extractor(writer2_2)
    f3_1  = Feature_Extractor(writer3_1)
    f3_2  = Feature_Extractor(writer3_2)
    pred  = Feature_Extractor(test_img)
    
    features = np.array([f1_1,f1_2,f2_1,f2_2,f3_1,f3_2])
    
    classifier = SVC()
    classifier.fit(features, [1,1,2,2,3,3])
    result = classifier.predict([pred])
    
    
    en = time.time()
    print("Time Taken : ",(en-st))
    print(result,correct[i])
    if(result==correct[i]):
        acc+=1
    i+=1
    print(i,acc, "Accuracy : ",acc*100/i,"%")
    
    
    

Time Taken :  14.422209978103638
[3] 3
1 1 Accuracy :  100.0 %
Time Taken :  14.078056573867798
[3] 3
2 2 Accuracy :  100.0 %
Time Taken :  14.637044668197632
[3] 3
3 3 Accuracy :  100.0 %
Time Taken :  14.349113702774048
[3] 3
4 4 Accuracy :  100.0 %
Time Taken :  14.696416139602661
[3] 3
5 5 Accuracy :  100.0 %
Time Taken :  14.118675708770752
[3] 3
6 6 Accuracy :  100.0 %
Time Taken :  14.523785591125488
[2] 3
7 6 Accuracy :  85.71428571428571 %
Time Taken :  15.08477520942688
[2] 3
8 6 Accuracy :  75.0 %
Time Taken :  14.605950832366943
[3] 3
9 7 Accuracy :  77.77777777777777 %
Time Taken :  15.568571329116821
[3] 3
10 8 Accuracy :  80.0 %
Time Taken :  15.521966457366943
[3] 3
11 9 Accuracy :  81.81818181818181 %
Time Taken :  14.514422416687012
[3] 3
12 10 Accuracy :  83.33333333333333 %
Time Taken :  14.182424545288086
[3] 3
13 11 Accuracy :  84.61538461538461 %


# Random Test


In [12]:
with open("forms.txt") as h:
    lines = h.readlines()[16:]

In [13]:
freq = np.zeros(680, dtype=int)
formWriter = {}
for i in lines:
    line = i.split(' ')
    key = line[1]
    if key in formWriter.keys():
        formWriter[key].append(line[0])
    else:
        formWriter[key] = [line[0]]

In [14]:
freq = [x for x, l in formWriter.items() if len(l)>2]

In [18]:
acc = 0
total = 0
for itr in range(100):
    keys = np.random.randint(len(freq), size=3)
    train = []
    test = []
    for k in keys:
        forms = formWriter[freq[k]]
        forms = [freq[k]+'-'+'{:03d}'.format(s)+'.jpg' for s in range(len(forms))]
        idxes = np.random.randint(len(forms), size=3)
        train.append(forms[idxes[0]])
        train.append(forms[idxes[1]])
        test.append(forms[idxes[2]])
    testWriter = np.random.randint(len(test))
    testImg = test[testWriter]
    
    trainFeatures = []
    Y = []
    for i in range(len(train)):
        img = cv2.imread('train/'+train[i], cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = cv2.resize(img,(img.shape[0]//4,img.shape[1]//4))
       
        f = Feature_Extractor(img)
        trainFeatures.append(f)
        Y.append(i//2)
    trainFeatures =  np.array(trainFeatures)
    classifier = SVC()
   
    classifier.fit(trainFeatures, Y)
    
    img = cv2.imread("train/"+testImg, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue
    img = cv2.resize(img,(img.shape[0]//4,img.shape[1]//4))
    testFeatures = Feature_Extractor(img)
    result = classifier.predict([testFeatures])
    if result[0] == testWriter:
        acc += 1
    total +=1
    print(len(testFeatures))
    print(total,acc, "Accuracy : ",acc*100/total,"%")
    print(result, testWriter)
    
    
    



883
1 1 Accuracy :  100.0 %
[2] 2
883
2 2 Accuracy :  100.0 %
[1] 1
883
3 3 Accuracy :  100.0 %
[0] 0
883
4 4 Accuracy :  100.0 %
[0] 0
883
5 5 Accuracy :  100.0 %
[1] 1
883
6 6 Accuracy :  100.0 %
[2] 2
883
7 7 Accuracy :  100.0 %
[2] 2
883
8 8 Accuracy :  100.0 %
[2] 2
883
9 9 Accuracy :  100.0 %
[0] 0
883
10 10 Accuracy :  100.0 %
[0] 0
883
11 11 Accuracy :  100.0 %
[0] 0
883
12 12 Accuracy :  100.0 %
[0] 0
883
13 13 Accuracy :  100.0 %
[0] 0
883
14 14 Accuracy :  100.0 %
[2] 2
883
15 14 Accuracy :  93.33333333333333 %
[2] 1
883
16 14 Accuracy :  87.5 %
[1] 2
883
17 14 Accuracy :  82.3529411764706 %
[1] 0
883
18 15 Accuracy :  83.33333333333333 %
[2] 2
883
19 16 Accuracy :  84.21052631578948 %
[2] 2
883
20 17 Accuracy :  85.0 %
[2] 2
883
21 18 Accuracy :  85.71428571428571 %
[0] 0
883
22 19 Accuracy :  86.36363636363636 %
[1] 1
883
23 20 Accuracy :  86.95652173913044 %
[0] 0
883
24 21 Accuracy :  87.5 %
[1] 1
883
25 22 Accuracy :  88.0 %
[0] 0
883
26 22 Accuracy :  84.61538461538461

KeyboardInterrupt: 

# Results 
### SVM :  Normal Windows , No Ali part  : 86%    (86/100)     , 83%
### KNN :  Normal Windows , No Ali part  : 74%    (74/100)
### SVM :  Normal Windows , No Ali part no local features (84/100)
### SVM :  Normal Windwos , No Ali part , Pca  :64% (7/11)